In [95]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

base_path = Path()
big_data = base_path.joinpath('eda_data.csv')
big_df = pd.read_csv(big_data)

big_df['x10'] = big_df['x10'].str.strip('%').astype(float) / 100
big_df['x6'] = big_df['x6'].str.replace(r'[\)\$\,]','')
big_df['x6'] = big_df['x6'].str.replace(r'\(','-').astype(float)
big_df.dropna(inplace=True, how='any')
display(big_df.head(10))

C:\Users\sebas\AppData\Local\Temp\ipykernel_9336\2418468216.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  big_df['x6'] = big_df['x6'].str.replace(r'[\)\$\,]','')
C:\Users\sebas\AppData\Local\Temp\ipykernel_9336\2418468216.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  big_df['x6'] = big_df['x6'].str.replace(r'\(','-').astype(float)


,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,y
0,-17.933519,6.559220,-14.452810,-4.732855,0.381673,2.563194,-1306.52,-89.394348,-28.454044,-16.201298,-0.0001,0.217010,9.729891,-0.786431,0.666146
1,-37.214754,10.774930,-15.384004,-0.077339,10.983774,-15.210206,-24.86,153.032652,-32.557736,69.675903,0.0000,-3.584908,35.727926,-0.985552,0.378411
2,0.330441,-19.609972,-9.167911,2.064124,12.071688,12.506141,-110.85,-141.437276,-20.794952,55.042604,0.0000,-3.991366,-9.283523,-3.394718,0.624498
3,-13.709765,-8.011390,6.759264,1.727615,-1.768382,24.039733,-324.43,51.039653,-7.046908,-31.424419,0.0001,7.908897,-2.891882,-2.690222,0.126622
4,-4.202598,7.076210,-26.004919,-4.269696,-3.414224,2.115989,1213.37,-31.046700,19.061182,-31.525515,-0.0001,0.846719,25.497480,3.516801,0.640025
5,12.570277,2.543883,25.673620,-0.842858,3.400531,37.176963,-1508.42,-98.956484,11.734073,100.654890,0.0000,15.359466,9.000782,1.929145,0.875990
6,17.949374,-35.406853,-25.164967,0.587439,-13.782894,-5.204895,-1431.44,-205.638274,16.399039,116.749751,0.0001,5.964466,15.851576,-0.819405,0.428533
7,38.645268,-21.871066,-74.405515,-9.707926,6.114417,-17.883435,1613.55,-239.151432,28.570116,-333.261028,0.0001,5.556269,-15.112181,2.480845,0.350310
8,35.279887,-2.004483,-12.071977,-1.020028,3.250741,-19.377852,1178.11,-108.229348,-29.886388,-126.434298,0.0000,4.085897,-10.558902,1.700618,0.612351
9,-31.756646,-5.499585,6.198889,-4.569550,13.902393,1.031500,-312.04,299.025699,24.338666,-25.001839,0.0000,4.204932,-5.320926,4.405564,0.494439


In [96]:
x = big_df.drop(columns=['y'])
y = big_df['y']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [99]:
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
r2 = r2_score(y_test, y_pred)
print(f'R2 score: {r2}')

R2 score: -0.001258513282838436


In [111]:
import torch, matplotlib.pyplot as plt
from torch.autograd import Variable

class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
        
    def forward(self, x):
        out = self.linear(x)
        return out
    
inputDim = 14
outputDim = 1
learningRate = 1.5e-6
epochs = 10_000

model = linearRegression(inputDim, outputDim)
if torch.cuda.is_available():
    model.cuda()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)

for epoch in range(epochs):
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train.values).float().cuda())
        labels = Variable(torch.from_numpy(y_train.values).float().cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train.values).float())
        labels = Variable(torch.from_numpy(y_train.values).float())
        
    optimizer.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    print('epoch {}, loss {}'.format(epoch, loss.item()))

with torch.no_grad():
    if torch.cuda.is_available():
        predicted = model(Variable(torch.from_numpy(x_test.values).float().cuda())).cpu().data.numpy()
    else:
        predicted = model(Variable(torch.from_numpy(x_test.values).float())).data.numpy()
    
print(f'PyTorch R2 score: {r2_score(y_test, predicted)}')
    

c:\Users\sebas\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([7976])) that is different to the input size (torch.Size([7976, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 0, loss 5984.755859375
epoch 1, loss 22278.09765625
epoch 2, loss 9503.009765625
epoch 3, loss 1234.71533203125
epoch 4, loss 14046.34765625
epoch 5, loss 10779.6962890625
epoch 6, loss 297.81396484375
epoch 7, loss 7367.095703125
epoch 8, loss 9932.384765625
epoch 9, loss 1157.0810546875
epoch 10, loss 2996.03955078125
epoch 11, loss 7797.11669921875
epoch 12, loss 2355.843017578125
epoch 13, loss 781.7142333984375
epoch 14, loss 5289.103515625
epoch 15, loss 3107.71240234375
epoch 16, loss 96.35897827148438
epoch 17, loss 3072.049560546875
epoch 18, loss 3188.325927734375
epoch 19, loss 224.46649169921875
epoch 20, loss 1471.5546875
epoch 21, loss 2726.712890625
epoch 22, loss 601.7913818359375
epoch 23, loss 530.7200927734375
epoch 24, loss 1993.5911865234375
epoch 25, loss 898.5009765625
epoch 26, loss 120.39700317382812
epoch 27, loss 1246.6348876953125
epoch 28, loss 994.401123046875
epoch 29, loss 46.83125305175781
epoch 30, loss 651.7377319335938
epoch 31, loss 905.452514